In [2]:
import time
import re
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

rings_all_link = "https://www.zales.com/rings/rings/c/0106050000?icid=RINGS_VIEWALL" 
driver.get(rings_all_link)


jewels_name_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "name", " " ))]//a'
jewels_price_now_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "price", " " ))]'
    
SCROLL_PAUSE_TIME = 0.5

only_first_n = 1000 # to avoid blocking of the ip, we only get 1000 objects 
count = 0

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    jewels_name = driver.find_elements_by_xpath(jewels_name_xpath)
    jewels_price_now = driver.find_elements_by_xpath(jewels_price_now_xpath)
    
    if len(jewels_name)/2 >= only_first_n: #
        break

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

name_list = []
price_now_list = []

name_list = [i.text for i in jewels_name if i.text]
price_now_list = [i.text for i in jewels_price_now if i.text]

pattern_ct = re.compile('((\d+(\/\d+)?)\s+CT\.\s+T\.W\.)|(\d+\.?\d*mm)')
ct_list = []
for i in name_list:
    maybe_match = pattern_ct.search(i)
    if maybe_match is None:
        match = ['Na']
    else:
        match = [maybe_match.group(0)]
    ct_list += match

pattern_price = re.compile('\$[0-9,\.]*') #
cost_list = []
for i in price_now_list:
    maybe_match = pattern_price.search(i)
    if maybe_match is None:
        match = ['Na']
    else:
        match = [maybe_match.group(0)]
    cost_list += match

pattern_discount = re.compile('(?<=\()(.*?)(?<=%)')
discount_list = []
for i in price_now_list:
    maybe_match = pattern_discount.search(i)
    if maybe_match is None:
        match = ['0']
    else:
        match = [maybe_match.group(0)]
    discount_list += match

    
with open('rings.txt', 'w') as f:
    #header 
    header = ['name','size','cost','discount', 'original_name', 'original_price']
    f.write(','.join(header) + '\n')

    #body
    for i in range(len(name_list)):
        f.write(re.sub(pattern_ct, '', name_list[i]).replace(',','').lstrip() + ', ')
        f.write(ct_list[i] + ', ')
        f.write(cost_list[i].replace(',','').lstrip() + ', ') #
        f.write(discount_list[i] + ', ')
        f.write(name_list[i].replace(',','').lstrip() + ', ')
        f.write(price_now_list[i].replace(',','').lstrip())
        f.write('\n')
        
driver.close()
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\franc\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache
